In [2]:
import os
import yaml
import sys
#read a yml file and return the output as a dictionary
def read_yml(file_name):
    with open(file_name, 'r') as stream:
        # load the yml file as a dictionary
        config = yaml.safe_load(stream)
    return config

config = read_yml("config_hyperpar.yml") 

In [3]:
type(config['no_pe'])

bool

In [4]:
# a function to get the number of gpus available

def get_gpus():
    os.system("nvidia-smi -L")
    gpus = os.popen("nvidia-smi -L").read().splitlines()
    return len(gpus)

In [5]:
n_gpu = get_gpus()

GPU 0: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-adb38e32-69a6-943c-1329-6cb1bea5607a)
GPU 1: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-48ea4460-f033-a344-3849-1e14791aef64)
GPU 2: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-75d67f86-0d17-1fce-b2c9-409c006ea79c)
GPU 3: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-54546aee-7861-497c-6260-921580b2fac3)
GPU 4: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-2daa1cb4-049c-1447-2cdc-f8d663a0b269)
GPU 5: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-a7dcef3d-372b-1f2c-1526-0d4e3625febf)
GPU 6: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-4865a397-0d06-2eca-f903-c77e15ddfabb)
GPU 7: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-25d5ce06-bdba-db56-1921-517b1c804a84)
GPU 8: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-8628b156-32f2-35bc-fce0-f03b7fd0902d)
GPU 9: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-e5df9d91-4acc-ac98-e6c0-a04c847f5990)


In [6]:
n_gpu

10

In [7]:
#import cartesian product
from itertools import product
list_of_keys = []
for key in config.keys():
    if isinstance(config[key], list):
        list_of_keys.append(key)

# create a cartesian product of the list of keys and the values inside the list
# and return the list of tuples
cartesian_product = list(product(*(config[key] for key in list_of_keys)))
print(cartesian_product)
# create a list of dictionaries
# each dictionary is a combination of the cartesian product of the list of keys and the values inside the list
list_of_dict = [dict(zip(list_of_keys, tup)) for tup in cartesian_product]
print(list_of_dict)

[('a 3D rendering of a Roman soldier', 12.0, 4, 29, True), ('a 3D rendering of a Roman soldier', 12.0, 4, 29, False), ('a 3D rendering of a Roman soldier', 12.0, 4, 1821, True), ('a 3D rendering of a Roman soldier', 12.0, 4, 1821, False), ('a 3D rendering of a Roman soldier', 12.0, 4, 3661, True), ('a 3D rendering of a Roman soldier', 12.0, 4, 3661, False), ('a 3D rendering of a Roman soldier', 12.0, 100, 29, True), ('a 3D rendering of a Roman soldier', 12.0, 100, 29, False), ('a 3D rendering of a Roman soldier', 12.0, 100, 1821, True), ('a 3D rendering of a Roman soldier', 12.0, 100, 1821, False), ('a 3D rendering of a Roman soldier', 12.0, 100, 3661, True), ('a 3D rendering of a Roman soldier', 12.0, 100, 3661, False), ('a 3D rendering of a Roman soldier', 5.0, 4, 29, True), ('a 3D rendering of a Roman soldier', 5.0, 4, 29, False), ('a 3D rendering of a Roman soldier', 5.0, 4, 1821, True), ('a 3D rendering of a Roman soldier', 5.0, 4, 1821, False), ('a 3D rendering of a Roman soldier

In [8]:
# write a script for each combination of parameters in list_of_dict
0 == False

True

In [9]:
# check the config dict and put the keys where their value is list in another list
from itertools import product
list_of_keys = []
for key in config.keys():
    if isinstance(config[key], list):
        list_of_keys.append(key)

# create a cartesian product of the list of keys and the values inside the list
# and return the list of tuples
cartesian_product = list(product(*(config[key] for key in list_of_keys)))
print(cartesian_product)
# create a list of dictionaries
# each dictionary is a combination of the cartesian product of the list of keys and the values inside the list
list_of_dict = [dict(zip(list_of_keys, tup)) for tup in cartesian_product]

# for each dictionary in the list_of_dict run the main.py script with the current combination of parameters
commands = []
for id_dict, dictionary in enumerate(list_of_dict):
    print(dictionary)
    # create the command to run the main.py script
    # add to the begining of the command the gpu number enumrated om
    command = "python main.py"
    # for each key in the dictionary append the key and the value to the command
    for key in config.keys():

        value = dictionary[key] if key in dictionary.keys() else config[key]
        if key == "output_dir":
            value = value + "/" + str(id_dict)
        # check the value if it is boolean and True then only add the key
        if isinstance(value, bool):
            if value == True:
                command += " --" + str(key)
        else:
            command += " --" + str(key) + " " + str(value)
    
   #  add the command to the list of commands
    commands.append(command)
# run each command on differnt gpu with based on cuda_visible_devices 
# and if the number of the commands are more than the number of gpu then wait and run the rest accoridnly
gpu_number = 0
with open("run_command.sh", "w") as text_file:
    for command in commands:
        # if the gpu number is less than the number of gpu then run the command on that gpu
        if gpu_number < n_gpu-1:
            gpu_number += 1
            command = "CUDA_VISIBLE_DEVICES=" + str(gpu_number) + " " + command + " &"
            # write the command to a bash file
            text_file.write(command)
            text_file.write("\n")
            
            
        # if the number of the commands is greater than the number of gpu then wait until the gpu is free and run the command
        else:
            text_file.write("echo 'waiting for gpus to be free' \n")
            text_file.write("wait \n")
            gpu_number = 0
    text_file.write("wait \n")
    text_file.write("echo 'all taskes are assigned'")
    text_file.write("wait \n")
    # zip everything inside config['output_dir'] 
    text_file.write("echo 'zipping'")
    text_file.write("\n")
    text_file.write("zip -r " + config['output_dir'] + "_zip " + config['output_dir'])
    text_file.write("\n")


    












[('a 3D rendering of a Roman soldier', 12.0, 4, 29, True), ('a 3D rendering of a Roman soldier', 12.0, 4, 29, False), ('a 3D rendering of a Roman soldier', 12.0, 4, 1821, True), ('a 3D rendering of a Roman soldier', 12.0, 4, 1821, False), ('a 3D rendering of a Roman soldier', 12.0, 4, 3661, True), ('a 3D rendering of a Roman soldier', 12.0, 4, 3661, False), ('a 3D rendering of a Roman soldier', 12.0, 100, 29, True), ('a 3D rendering of a Roman soldier', 12.0, 100, 29, False), ('a 3D rendering of a Roman soldier', 12.0, 100, 1821, True), ('a 3D rendering of a Roman soldier', 12.0, 100, 1821, False), ('a 3D rendering of a Roman soldier', 12.0, 100, 3661, True), ('a 3D rendering of a Roman soldier', 12.0, 100, 3661, False), ('a 3D rendering of a Roman soldier', 5.0, 4, 29, True), ('a 3D rendering of a Roman soldier', 5.0, 4, 29, False), ('a 3D rendering of a Roman soldier', 5.0, 4, 1821, True), ('a 3D rendering of a Roman soldier', 5.0, 4, 1821, False), ('a 3D rendering of a Roman soldier